**Datenerhebung**

Die Daten wurden von Spotify erhoben, die diverse Analysedaten zu einer großen Anzahl von Liedern zur Verfügung stellen. Mithilfe einer HTTP-GET kann eine Anfrage an die Server von Spotify geschickt werden und man erhält als Antwort diverse Daten zu den Liedern. Ziel ist einen Datensatz zu erheben mit Liedern, die von Spotify zu einem bestimmten Genre zugeordnet werden. Zusätzlich werden zu jedem Lied, weitere Analyse Daten erhoben. Dadurch soll ein Modell entwickelt werden, welches unbekannten Lieder automatisch zu einem Genre zuordnen kann. 

Zuerst werden die beiden Module *Pandas* und *Requests* geladen. Pandas dient zur Erstellung und Bearbeitung von Tabellen. Das Requests Modul wird für die HTTP-Anfragen genutzt.

In [20]:
import requests
import pandas as pd

Das *header*-Objekt muss immer im header bei den HTTP-Anfragen mitübergeben werden und beinhaltet eine Authentifizierung Token. Der Token wird von Spotify generiert und ist für eine Stunde gültig, bis ein neuer von Spotify generiert werden muss. 

In [21]:
token = "Bearer BQCJ3AHkRf1iIgQg_7Sli8qzEeAfApJ8753UhE9KExP7mTFMeGdnZIsKA_ybGEyqfjmbkRLaGJhBXuhkysMn8042_Kskhg11WoEDsSeUJqqHz3dm3PMXSwyThcOHC6szXLqO0rHsw9iCQQH1-pR6IChqzuwZsvTxgLcLWzUkMbcchK4"
header = {
    "Authorization": token
}

Die Spotify API bietet die Möglichkeit, sich Informationen über mehrere Playlisten zu beschaffen, die in einer bestimmten Kategorie fallen. Die Liste *genre* beinhaltet die IDs von den Kategorien, wodurch sich zum Beispiel Informationen abrufen lassen über alle Playlisten, die von Spotify erstellt, die in die Kategorie Pop fallen. Es fällt auf, dass die ID für das Metal Genre anders aufgebaut ist als die anderen IDs der Genres und anstatt aus dem Wort „metal“, aus einer Kombination aus Zahlen und Buchstaben besteht.

In [22]:
genre = ["pop","hiphop","edm_dance","rock","party","indie_alt","alternative","0JQ5DAqbMKFDkd668ypn6O"]

Über eine HTTP-GET anfrage lassen sich weitere Informationen zu den einzelnen Kategorien von Spotify beschaffen [[Kategorien](https://developer.spotify.com/console/get-browse-categories/), Abrufdatum: 11.07.2022]


In [23]:
%%time
rows = []
for i in genre:
    gern = i
    
    get_playlistID = f"https://api.spotify.com/v1/browse/categories/{gern}/playlists"
    playlistID = requests.get(get_playlistID,headers=header)
    if playlistID.status_code == 200:
        playlistID = playlistID.json()
    for j in playlistID["playlists"]["items"]:
        rows.append({"id":j["id"],"name":j["name"],"gerne":i})
    

TypeError: 'NoneType' object is not subscriptable

Die gesammelten Playlisten lassen sich in einen Datenframe umformen und durch, das Gruppieren nach Genre, lässt sich erkennen, dass zu jedem Genre nahezu gleichviele Playlisten gibt. 


In [24]:
df = pd.DataFrame(rows)

In [25]:
df.groupby("gerne").size()

gerne
0JQ5DAqbMKFDkd668ypn6O    17
alternative               18
edm_dance                 20
hiphop                    14
indie_alt                 20
party                      8
pop                       20
rock                      20
dtype: int64

In [26]:
#Umbennen des Genres für Playlisten in der Kategorie Metal
df.loc[df["gerne"] == "0JQ5DAqbMKFDkd668ypn6O","gerne"] = "metal"
df.groupby("gerne").size()

gerne
alternative    18
edm_dance      20
hiphop         14
indie_alt      20
metal          17
party           8
pop            20
rock           20
dtype: int64

In [27]:
rows = []
#Parameter für die Abfrage, da nur der Titel eines Liedes und die ID von interesse ist
params = {
    "fields" : "items(track(name,id))"
}

Mithelfe der gesammelten Informationen über die Playlisten, lassen sich mit einer weiteren HTTP-GET anfrage Informationen über den Inhalt der Playlisten beschaffen. Die Titel und IDs der Lieder werden wieder in einer Liste zwischengespeichert. Da die Lieder sich immer in einer Playlist zu einem bestimmten Genre befanden, können so die einzelnen Lieder einem Genre zugewiesen werden. 

In [28]:
%%time
for i in range(0,len(df)):
    play_ID = df["id"][i]
    url = f"https://api.spotify.com/v1/playlists/{play_ID}/tracks"
    
    #fetchdata
    x = requests.get(url,params=params,headers=header)
    if x.status_code != 200:
        print(f"Fehlercode: {x.status_code}")
        break
    x_dict = x.json()
    
    #Save Data
    for j in x_dict["items"]:
        try:
            rows.append({"id":j["track"]["id"],"titel":j["track"]["name"],"genre":df["gerne"][i]})
        except TypeError:
            pass

CPU times: total: 2.72 s
Wall time: 40.3 s


In [29]:
df = pd.DataFrame(rows)
df.shape

(10410, 3)

In [30]:
df.head()

,id,titel,genre
0,2cg9zDi0rvj3HjKxvv2tvY,Follow,pop
1,2q7jpVfCtcuRwcur9nQeuC,PARADISE,pop
2,4zN21mbAuaD0WqtmaTZZeP,Ferrari,pop
3,3a0F2N6rXJSibQTp77xH4f,Ohne Benzin,pop
4,75FEaRjZTKLhTrFGsfMUXR,Running Up That Hill (A Deal With God),pop


Lieder können sich in mehreren Playlisten befinden, auch zu anderen Genres, können die Lieder nicht eindeutig einem Genre zugewiesen werden. Um die Duplikate zu entfernen und die Lieder eindeutig einem Genre zuzuweisen, wird die Tabelle nach den IDs gruppiert und nach der Häufigkeit des Auftretens in den Genres sortiert. Ein Lied das zum Beispiel in drei Playlisten zum Genre Metal zu finden war und in zwei Playlisten zum Genre Rock, wird so dem Genre Metal zugewiesen.

In [31]:
df = df.groupby("id")["titel","genre"].max().reset_index()
df.shape

C:\Users\Moritz\AppData\Local\Temp\ipykernel_8988\211789912.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby("id")["titel","genre"].max().reset_index()


(8642, 3)

In [32]:
df.head()

,id,titel,genre
0,0038p5duwZ3euoTDxmizmF,I Don't Know You,metal
1,003vvx7Niy0yvhvHt4a68B,Mr. Brightside,rock
2,009RK7haVVIsa0seuyNSFR,Sous le soleil de Bodega,rock
3,00CKqQSBhAr2gePvyIzk0P,Even Angels Cry,metal
4,00FRYCC99BHS1ewIqteTwh,hunger,indie_alt


Durch die IDs der einzelnen Lieder lassen sich analytische Informationen zu jedem Lied beschaffen, die von Spotify zur Verfügung gestellt werden. Zu den Informationen gehören unteranderem die Länge, die Tonart, das Tempo und der Anteil an gesprochenen Worten eines Liedes. 


In [33]:
%%time
track_id = df["id"]
track_featured_list = []

for i in track_id:
    url_track_feature = f"https://api.spotify.com/v1/audio-features/{i}"
    get_track_features = requests.get(url_track_feature,headers=header)
    
    if get_track_features.status_code != 200:
        print(f"Fehlercode: {get_track_features.status_code}")
        continue
    
    track_features = get_track_features.json()
    
    track_features.pop("type",None)
    track_features.pop("uri",None)
    track_features.pop("track_href",None)
    track_features.pop("analysis_url",None)
    
    track_featured_list.append(track_features)

Fehlercode: 404
Fehlercode: 404
CPU times: total: 3min 11s
Wall time: 23min 5s


Die Informationen werden erneut in einer Liste zwischengespeichert und anschließend in einer Tabelle umgeformt.

In [34]:
df_temp = pd.DataFrame(track_featured_list)

In [35]:
df_temp.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.421,0.949,5,-2.215,0,0.0932,0.000189,0.009940,0.4480,0.270,148.048,0038p5duwZ3euoTDxmizmF,221988,4
1,0.352,0.911,1,-5.230,1,0.0747,0.001210,0.000000,0.0995,0.236,148.033,003vvx7Niy0yvhvHt4a68B,222973,4
2,0.668,0.913,5,-4.449,0,0.0573,0.142000,0.000007,0.1400,0.594,102.841,009RK7haVVIsa0seuyNSFR,206893,4
3,0.563,0.414,10,-10.914,1,0.0270,0.027600,0.000000,0.1550,0.193,132.107,00CKqQSBhAr2gePvyIzk0P,287293,4
4,0.586,0.740,2,-6.132,1,0.0398,0.311000,0.000353,0.0878,0.473,145.871,00FRYCC99BHS1ewIqteTwh,191297,4


Die Tabelle mit den Liedern zu einem Genre und die Tabelle mit den analytischen Daten zu den Liedern, lassen sich zu einer Tabelle anhand der ID zusammenführen und kann anschließend als CSV-Datei exportiert werden.

In [36]:
df_dump = df.merge(df_temp,on="id",how="inner")
print(df.shape)
print(df_temp.shape)
print(df_dump.shape)

(8642, 3)
(8640, 14)
(8640, 16)


In [37]:
df_dump = df_dump.drop_duplicates(subset=["id","genre"],ignore_index=True)
df_dump.shape

(8640, 16)

In [38]:
df_dump.to_csv("music_data.csv",index=False)